In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from	sklearn.feature_extraction.text	import	TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

from ast import literal_eval

import warnings
warnings.filterwarnings('ignore')

In [2]:
clean_df = pd.read_csv('/content/drive/MyDrive/RecomSysData/movies_metadata_clean.csv')
clean_df

,title,genres,runtime,vote_average,vote_count,year
0,Toy Story,"['Animation', 'Comedy', 'Family']",81.0,7.7,5415.0,1995.0
1,Jumanji,"['Adventure', 'Fantasy', 'Family']",104.0,6.9,2413.0,1995.0
2,Grumpier Old Men,"['Romance', 'Comedy']",101.0,6.5,92.0,1995.0
3,Waiting to Exhale,"['Comedy', 'Drama', 'Romance']",127.0,6.1,34.0,1995.0
4,Father of the Bride Part II,['Comedy'],106.0,5.7,173.0,1995.0
...,...,...,...,...,...,...
45461,Subdue,"['Drama', 'Family']",90.0,4.0,1.0,0.0
45462,Century of Birthing,['Drama'],360.0,9.0,3.0,2011.0
45463,Betrayal,"['Action', 'Drama', 'Thriller']",90.0,3.8,6.0,2003.0
45464,Satan Triumphant,[],87.0,0.0,0.0,1917.0


In [3]:
orig_df = pd.read_csv('/content/drive/MyDrive/RecomSysData/movies_metadata.csv', low_memory= True)
orig_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [4]:
clean_df['overview'], clean_df['id'] = orig_df['overview'], orig_df['id']

In [5]:
clean_df.head()

,title,genres,runtime,vote_average,vote_count,year,overview,id
0,Toy Story,"['Animation', 'Comedy', 'Family']",81.0,7.7,5415.0,1995.0,"Led by Woody, Andy's toys live happily in his ...",862
1,Jumanji,"['Adventure', 'Fantasy', 'Family']",104.0,6.9,2413.0,1995.0,When siblings Judy and Peter discover an encha...,8844
2,Grumpier Old Men,"['Romance', 'Comedy']",101.0,6.5,92.0,1995.0,A family wedding reignites the ancient feud be...,15602
3,Waiting to Exhale,"['Comedy', 'Drama', 'Romance']",127.0,6.1,34.0,1995.0,"Cheated on, mistreated and stepped on, the wom...",31357
4,Father of the Bride Part II,['Comedy'],106.0,5.7,173.0,1995.0,Just when George Banks has recovered from his ...,11862


In [6]:
clean_df['overview']	=	clean_df['overview'].fillna('')

In [7]:
sample = clean_df.sample(n = 30000, random_state= 42).reset_index()
sample

,index,title,genres,runtime,vote_average,vote_count,year,overview,id
0,43526,Small Crimes,"['Drama', 'Comedy', 'Thriller', 'Crime']",95.0,5.8,55.0,2017.0,"A disgraced former cop, fresh off a six-year p...",411405
1,6383,Up the Sandbox,"['Drama', 'Comedy']",97.0,7.3,2.0,1972.0,"A young wife and mother, bored with day-to-day...",42492
2,3154,Bad Lieutenant,"['Crime', 'Drama']",96.0,6.9,162.0,1992.0,"While investigating a young nun's rape, a corr...",12143
3,10146,Satan's Little Helper,"['Horror', 'Romance', 'Comedy']",100.0,5.0,42.0,2004.0,A naïve young boy unknowingly becomes the pawn...,9976
4,9531,Sitcom,"['Comedy', 'Drama', 'Thriller']",80.0,6.4,27.0,1998.0,The adventures of an upper-class suburban fami...,46761
...,...,...,...,...,...,...,...,...,...
29995,34303,Red Rose,"['Drama', 'Horror', 'Thriller']",125.0,0.0,0.0,1980.0,Anand lives a wealthy lifestyle in Bombay in a...,366120
29996,29610,Emmanuelle in Bangkok,"['Drama', 'Romance']",87.0,4.4,7.0,1976.0,Emanuelle is sent to Bangkok on a journalism a...,39965
29997,938,The 39 Steps,"['Action', 'Thriller', 'Mystery']",86.0,7.4,217.0,1935.0,"While on vacation in London, Canadian Richard ...",260
29998,26662,The Flesh and the Fiends,['Horror'],97.0,5.4,7.0,1960.0,Edinburgh surgeon Dr. Robert Knox requires cad...,52203


# Creating TF-IDF matrix:

In [8]:
tfidf	=	TfidfVectorizer(stop_words='english', #max_features= 3000
                        )
tfidf_matrix	=	tfidf.fit_transform(sample['overview'])
tfidf_matrix.shape

(30000, 61962)

# Compute Similarity:

In [9]:
cosine_similarity = linear_kernel(tfidf_matrix, tfidf_matrix)

In [10]:
indices	=	pd.Series(sample.index,	index=sample['title']).drop_duplicates()
indices

title
Small Crimes                    0
Up the Sandbox                  1
Bad Lieutenant                  2
Satan's Little Helper           3
Sitcom                          4
                            ...  
Red Rose                    29995
Emmanuelle in Bangkok       29996
The 39 Steps                29997
The Flesh and the Fiends    29998
You and Your Stupid Mate    29999
Length: 30000, dtype: int64

# Recommender Function:

In [11]:
def content_recommender(title, cosine_similarity = cosine_similarity, sample = sample, indices = indices):

  # get movie index:
  idx = indices[title]

  # get pairwise similarities
  sim_scores = list(enumerate(cosine_similarity[idx]))

  # sort similarities
  sim_scores	=	sorted(sim_scores,	key=lambda	x:	x[1],	reverse=True)

  # get the 10 most similar movies:
  sim_scores = sim_scores[1:11]

  # get the movies indices
  movies_indices = [i[0] for i in sim_scores]

  # return movies titles
  return sample['title'].iloc[movies_indices]
  



In [12]:
one_movie = sample.title[0]
one_movie

'Small Crimes'

In [13]:
content_recommender('Toy Story')

18195                                     Toy Story 3
26155                                     Toy Story 2
24720                          The 40 Year Old Virgin
3093                                        Small Fry
29880                     Andy Hardy's Blonde Trouble
7492     Superstar: The Life and Times of Andy Warhol
28269                Andy Kaufman Plays Carnegie Hall
21089    Andy Peters: Exclamation Mark Question Point
29521                                       The Champ
27353                           Rebel Without a Cause
Name: title, dtype: object

In [15]:
content_recommender('The Lion King')

3560                         The Lion King 1½
20662          The Lion King 2: Simba's Pride
2341                             African Cats
22533       Massaï, les guerriers de la pluie
21701                              Sour Grape
13180                              White Lion
13416    Michael Jackson: Life of a Superstar
27299                       His Name was King
23323                   The Little Bear Movie
27248            Michael Jackson: Number Ones
Name: title, dtype: object